# Production stage: use files 0-6 for model traning

In [1]:
!ls raw_data


data_gathered_at_0.csv	   data_gathered_at_3.csv  initial_training_data.csv
data_gathered_at_1.csv	   data_gathered_at_4.csv  test_data.csv
data_gathered_at_2.csv	   data_gathered_at_5.csv  validation_test_data.csv
data_gathered_at_2_bk.csv  data_gathered_at_6.csv


In [2]:
!mkdir accum_data

mkdir: cannot create directory ‘accum_data’: File exists


In [3]:
!cp raw_data/initial_training_data.csv accum_data/accumulated_data.csv


In [5]:
!ls -l accum_data/

total 2620
-rw-r--r-- 1 root root 2679360 May 30 12:42 accumulated_data.csv


In [6]:
import pandas as pd
import numpy as np
import os

from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical


raw_data_path = 'raw_data/'
accum_data_path = 'accum_data/'
accum_file_name = accum_data_path+'accumulated_data.csv'

def create_model():
    model = models.Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPool2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(optimizer=Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


def prepare_data(data):
    """ Prepare data for modeling 
        input: data frame with labels und pixel data
        output: image and label array """
    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['Emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'Pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label


def data_to_tf_data(df):
    image_array, image_label = prepare_data(df)
    images = image_array.reshape((image_array.shape[0], 48, 48, 1))
    images = images.astype('float32')/255
    labels = to_categorical(image_label)
    return images, labels


In [7]:
def add_to_accum_data(nmb):
    accum_data = pd.read_csv(accum_file_name)
    next_data_file = raw_data_path+'data_gathered_at_' + str(nmb)+'.csv'
    add_data = pd.read_csv(next_data_file)
    accum_data = accum_data.append(add_data, ignore_index = True)
    accum_data.to_csv(accum_file_name)


In [17]:
nmb = 1
add_to_accum_data(nmb)


In [18]:
train = pd.read_csv(accum_file_name)
train_images, train_labels = data_to_tf_data(train)

val = pd.read_csv(raw_data_path+'validation_test_data.csv')
val_images, val_labels = data_to_tf_data(val)


model = create_model()
class_weight = dict(zip(range(0, 7), (((train['Emotion'].value_counts()).sort_index())/len(train['Emotion'])).tolist()))
history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    class_weight = class_weight,
                    epochs=12,
                    batch_size=64)

df = pd.read_csv(raw_data_path+'test_data.csv')
test_images, test_labels = data_to_tf_data(df)
test_loss, test_acc = model.evaluate(test_images, test_labels)

/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/12
9/9 [==============================] - 4s 424ms/step - loss: 0.3199 - accuracy: 0.2505 - val_loss: 1.8804 - val_accuracy: 0.2449
Epoch 2/12
9/9 [==============================] - 3s 411ms/step - loss: 0.3024 - accuracy: 0.2635 - val_loss: 1.8988 - val_accuracy: 0.2449
Epoch 3/12
9/9 [==============================] - 3s 421ms/step - loss: 0.2964 - accuracy: 0.2635 - val_loss: 1.9546 - val_accuracy: 0.2449
Epoch 4/12
9/9 [==============================] - 3s 392ms/step - loss: 0.2921 - accuracy: 0.2635 - val_loss: 1.8976 - val_accuracy: 0.2449
Epoch 5/12
9/9 [==============================] - 3s 409ms/step - loss: 0.2916 - accuracy: 0.2635 - val_loss: 1.9475 - val_accuracy: 0.2449
Epoch 6/12
9/9 [==============================] - 3s 405ms/step - loss: 0.2929 - accuracy: 0.2635 - val_loss: 1.8603 - val_accuracy: 0.2449
Epoch 7/12
9/9 [==============================] - 3s 413ms/step - loss: 0.2891 - accuracy: 0.2635 - val_loss: 1.9650 - val_accuracy: 0.2449
Epoch 8/12
9/9 [====

In [11]:
from tensorflow import keras

model.save("initial_model")


INFO:tensorflow:Assets written to: initial_model/assets


In [12]:
!dvc add initial_model/


Adding...                                                                       
!
Computing file/dir hashes (only done once)            |0.00 [00:00,      ?md5/s]
                                                                                
!
                                                      |0.00 [00:00,       ?it/s]
                                                                                
!
  0%|          |Saving files                          0/4 [00:00<?,     ?file/s]
                                                                                
!
  0%|          |.oPt3T5N5yhYqrbVfAjNGGY.tmp           0/1 [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.fqFQgoUXRF38Q4bm9UFg26.tmp     0.00/164k [00:00<?,       ?it/s]
                                                                                
!
  0%|          |.HMLJeamqTmrvkHcTgrkKLr.tmp    0.00/3.83M [00:00<?,       ?it/s]
                

In [13]:
!git add initial_model.dvc

In [14]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 2 commits.
  (use "git push" to publish your local commits)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   initial_model.dvc

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   FinalProj2_Initial.ipynb
	modified:   FinalProj2_production.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	Untitled.ipynb
	accum_data/
	mlruns/



In [15]:
!git add FinalProj2_Initial.ipynb FinalProj2_production.ipynb

In [16]:
!git commit -m "Model trained wit data at_0"

[master d278e62] Model trained wit data at_0
 3 files changed, 384 insertions(+), 245 deletions(-)
